# Test Task: Parsing Resumes in Ukrainian
**Objective**: develop a program capable of automatically parsing resumes in Ukrainian to extract key data in a structured format. This task aims to assess skills in processing natural language texts, data extraction, and developing basic NLP solutions.

### Requirements
- **Input data**: 10-15 resumes in text format (TXT, PDF, or DOCX), all written in Ukrainian.
- **Information to extract**: candidate’s name, contact details, work experience (position, company, dates, responsibilities), education (institution, specialization, graduation year), and skills.
- **Execution steps**: use regular expressions, NLP libraries, or other text parsing methods.
- **Program requirements**: it should generalize to various resumes and handle different data formats.
- **Technologies**: python with libraries like `spaCy`, `re`, `pandas`, or `pdfplumber`.

# Theory

**Scapy** is a powerful Python library for Natural Language Understanding (NLU) that offers fast processing, excellent optimization for CPUs, and a large supportive community. It allows the creation of custom pipelines with tools like Named Entity Recognition (NER) to identify key entities in text. Spacy also provides easy integration with many languages and is highly customizable for specific tasks, making it ideal for dialogue systems.

Alternatives like **Stanza** and **Flair** also offer valuable features. **Stanza** supports over 60 languages, including Ukrainian, and provides detailed linguistic information, making it a great option for multilingual systems. **Flair**, while not as fast as Spacy, offers excellent flexibility for training custom models, particularly useful for classification tasks using PyTorch. Each tool has its strengths, so it’s best to experiment with them to find the right fit for your project.

# Instructions

1. Set the path to the folder containing the CVs (scroll to **Mount Google Drive and Set Paths**) you want to process. Make sure the folder is accessible from your Google Drive.
2. Run the Script: execute the entire script to process the CVs in the specified folder.
3. Check Results:
To view the results, scroll to the "**Test on one resume**" section, or open the generated file:
   1. For one CV: *resume_output.json*
   2. For all CVs in the folder: `output.json`

# Table of Contents

>[Install Required Packages](#scrollTo=oABGql13C3yV)

>[Mount Google Drive and Set Paths](#scrollTo=MW5gy_5HC-Gv)

>[Read files (pdf, docx, txt)](#scrollTo=882K4jn5DDP4)

>[Get name](#scrollTo=O9k44JF3DiLO)

>[Get contact information (number, email)](#scrollTo=Z6oGEh9jDcD5)

>[Get experience (position, company, dates of employment, key responsibilities)](#scrollTo=Za8eign_Drzg)

>[Get education (institution, specialization, graduation year)](#scrollTo=murmAobTDxbP)

>[Get skills (list of technical and soft skills)](#scrollTo=mRnjAd_1EEOG)

>[Parse resume](#scrollTo=x4rNWhWjEMqk)

>[Test on one resume](#scrollTo=IFzoJghzEmY5)

>[Parse all resumes from the folder](#scrollTo=QOnskjoiExXX)

>[Made by](#scrollTo=ZVm2qZ9cODRj)



# Install Required Packages

In [1]:
!pip install spacy pandas pdfplumber python-docx
!python -m spacy download uk_core_news_lg

import pdfplumber
import docx
import re
import json
import spacy
import pandas as pd
import os
from spacy.matcher import Matcher
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.2/231.2 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Mount Google Drive and Set Paths

In [2]:
drive.mount("/content/drive")

folder_path = "/content/drive/MyDrive/Colab Notebooks/files/CVs"
file_path = "/content/drive/MyDrive/Colab Notebooks/files/CVs/CV_Designer.pdf"

nlp = spacy.load("uk_core_news_lg")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read files (pdf, docx, txt)

In [3]:
def read_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Get name

1. Initialize matcher: set up the Matcher with spaCy’s vocabulary to detect Ukrainian name patterns.
2. Define patterns: create patterns for common Ukrainian name structures. These patterns are intended to capture both first-last and first-middle-last name structures.
3. Match & extract name: process the text’s initial lines to identify probable name positions. If no entity is detected, use regex to identify capitalized words.
4. Return first match: return the first detected name, prioritizing spaCy NER over the custom matcher if both are present.

### TODO:

1. Handle cases where initials are present.
2. Expand to recognize single-word names or names with hyphens.

In [4]:
matcher = Matcher(nlp.vocab)

patterns = [
    [{'POS': 'PROPN'}, {'POS': 'PROPN'}],          # First Last
    [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First Middle Last
]

matcher.add("NAME", patterns)

def extract_name_ukrainian(text):
    if isinstance(text, spacy.tokens.Doc):
        text = text.text

    initial_lines = "\n".join(text.splitlines()[:3])
    doc = nlp(initial_lines)

    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    if names:
        return names[0]
    else:
        fallback_name_pattern = re.compile(r'\b[А-ЯЇЄІҐ][а-яїєіґ]+(?: [А-ЯЇЄІҐ]\.?| [А-ЯЇЄІҐ][а-яїєіґ]+)?\b')
        name_match = fallback_name_pattern.search(initial_lines)
        return name_match.group(0) if name_match else None

    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

    return None

In [5]:
sample_text_1 = """
Ярослав П
UX/UI Designer | Product Designer
Маю значнии досвід роботи в UX/UI дизаи ні...
"""

sample_text_2 = """
PYTHON FULL-STACK РОЗРОБНИК
Микола П
Я маю досвід роботи в розробці повного циклу програмного забезпечення...
"""

sample_text_3 = """
Ваховська Віра
(Навички & Проєкти & Мови & Більше)
vahovskavm2003@gmail.com • @vera_vah • LinkedIn • GitHub • 0969359340
"""

# Extract names
name1 = extract_name_ukrainian(sample_text_1)
name2 = extract_name_ukrainian(sample_text_2)
name3 = extract_name_ukrainian(sample_text_3)

print("Extracted Name 1:", name1)
print("Extracted Name 2:", name2)
print("Extracted Name 3:", name3)

Extracted Name 1: Ярослав П
Extracted Name 2: Микола П
Extracted Name 3: Ваховська Віра


# Get contact information (number, email)

1. Define regex patterns: use regex patterns for phone numbers and emails.
2. Extract phone numbers: match phone patterns, remove whitespace and return unique values.
3. Extract emails: match standard email formats and return unique values.
4. Combine results: return both phones and emails, if available, as a structured dictionary.

In [6]:
def extract_phone_numbers(text):
    phone_pattern = re.compile(
        r'\+?3?8?\s?\(?0\d{2}\)?\s?\d{3}[-\s]?\d{2}[-\s]?\d{2}'
    )
    phones = phone_pattern.findall(text)
    return [phone.strip() for phone in phones]

def extract_emails(text):
    email_pattern = re.compile(
        r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    )
    return email_pattern.findall(text)

def extract_contact_info(text):
    if isinstance(text, spacy.tokens.Doc):
        text = text.text

    phones = extract_phone_numbers(text)
    emails = extract_emails(text)

    phones = list(set(phones))
    emails = list(set(emails))

    return {
        "phones": phones if phones else None,
        "emails": emails if emails else None
    }

In [7]:
sample_resume_1 = """
Ярослав П
UX/UI Designer | Product Designer
vahovskavm2003@gmail.com • @vera_vah • LinkedIn • GitHub • 0969359340
"""

sample_resume_2 = """
Микола П
Full-Stack Developer
Контакти: m.pavlenko@gmail.com, +380639562147
"""

sample_resume_3 = """
Андрій К
Інженер-розробник
Email: andriy.k@example.com • andriy202.k@gmail.com
"""

result_1 = extract_contact_info(sample_resume_1)
result_2 = extract_contact_info(sample_resume_2)
result_3 = extract_contact_info(sample_resume_3)

print("Результати для резюме 1:", result_1)
print("Результати для резюме 2:", result_2)
print("Результати для резюме 3:", result_3)

Результати для резюме 1: {'phones': ['0969359340'], 'emails': ['vahovskavm2003@gmail.com']}
Результати для резюме 2: {'phones': ['+380639562147'], 'emails': ['m.pavlenko@gmail.com']}
Результати для резюме 3: {'phones': None, 'emails': ['andriy202.k@gmail.com', 'andriy.k@example.com']}


# Get experience (position, company, dates of employment, key responsibilities)

1. Search keywords: look for keywords indicating an experience section.
2. Define job pattern: use regex to match positions, companies, and employment dates.
3. Extract responsibilities: match bullet points following each date, indicating responsibilities.
4. Format results: return experience entries as structured data with fields for position, company, dates, and responsibilities.

### TODO

1. Adjust to handle variations in date formats.
2. Remove "\n".
3. Think about making it more flexible: without defying the exact structure

In [8]:
def extract_experience(text):
    experiences = []
    keywords = r'(досвід|проєкти|застосунок|додаток|система|сайт)'

    start_idx = re.search(keywords, text, re.IGNORECASE)
    if not start_idx:
        return None

    start_idx = start_idx.start()
    text_section = text[start_idx:]

    job_pattern = re.compile(r"([A-Za-zА-Яа-яІЇҐЄіїґє\s]+)\n([A-Za-zА-Яа-яІЇҐЄіїґє\s]+)\n([А-ЯІЇҐЄ][a-zа-яіїґє]+\s\d{4}\s?-\s?(?:[А-ЯІЇҐЄ][a-zа-яіїґє]+\s\d{4}|(?:Дотепер|Теперішній час))?)", re.MULTILINE)
    matches = job_pattern.findall(text_section)

    for match in matches:
        position, company, dates = match[0].strip(), match[1].strip(), match[2].strip()

        responsibilities = extract_responsibilities(text_section, dates)

        experiences.append({
            'position': position,
            'company': company,
            'dates': dates,
            'responsibilities': responsibilities
        })

    return experiences

# Helper function to extract responsibilities for each position
def extract_responsibilities(text, dates):
    responsibilities = []

    section_start = text.find(dates)
    section_end = text.find("\n\n", section_start)
    if section_end == -1:
        section_end = len(text)
    responsibilities_section = text[section_start + len(dates):section_end]

    resp_pattern = re.compile(r'•\s?(.*?)(?=\n•|\n$)', re.DOTALL)
    resp_matches = resp_pattern.findall(responsibilities_section)

    if resp_matches:
        responsibilities = [match.strip() for match in resp_matches if match.strip()]
    else:
        additional_pattern = re.compile(r"(обов'язки:?\s*|)(.*)", re.IGNORECASE | re.DOTALL)
        additional_match = additional_pattern.search(responsibilities_section)
        if additional_match:
            responsibilities_text = additional_match.group(2).strip()
            responsibilities.append(responsibilities_text)

    return responsibilities

In [9]:
resume_text = """
Ярослав П
досвід
UX/UI Дизайнер
TechVega Solutions
Червень 2022 - Дотепер
• Провів юзабіліті-тестування та інтерв'ю з користувачами
• Розробляв каркаси, прототипи та фінальні високоякісні дизайни
• Співпрацював з інженерами для впровадження адаптивних дизаїнів

UI/UX Дизайнер
CreativeForge Studio
Лютий 2020 - Травень 2022
Як UI/UX дизайнер у CreativeForge Studio, я займався створенням дизайну для
додатків з управління проєктами, спортивних платформ та новинних порталів. Мої
обов’язки включали проведення досліджень користувачів, створення інтерактивних
макетів та розробку елементів інтерфейсу, що підвищували зручність користувачів.

Додаток для управління запасами
Січень 2021 - Листопад 2022
Full-Stack Розробник TechSolutions
Опис: Додаток для управління запасами, який допомагає
підприємствам контролювати наявність товарів,
здійснювати замовлення та аналізувати продажі. Інтерфейс
дозволяє користувачам відслідковувати запаси в режимі
реального часу, отримувати сповіщення про низькі запаси
та генерувати звіти.
Технології: Python, Django, PostgreSQL, Redis, JavaScript,
React, Redux, CSS, Stripe, AWS, Docker, CircleCI, Pytest.
Обовʼязки:
• Розробив API для деяких коефіцієнтів;
• Працював над функціональністю та дизайном статистики;
• Реалізував функцію підписки за допомогою Stripe;
• Переробив сторінку адміністратора;
• Виправив помилки на бекенді та фронтенді;
• Створив юніт-тести;
• Проводив рев’ю коду;
• Брав участь у Scrum-зустрічах
"""

experience = extract_experience(resume_text)
print(f"Final experience: {experience}")

Final experience: [{'position': 'UI Дизайнер', 'company': 'TechVega Solutions', 'dates': 'Червень 2022 - Дотепер', 'responsibilities': ["Провів юзабіліті-тестування та інтерв'ю з користувачами", 'Розробляв каркаси, прототипи та фінальні високоякісні дизайни']}, {'position': 'UX Дизайнер', 'company': 'CreativeForge Studio', 'dates': 'Лютий 2020 - Травень 2022', 'responsibilities': ['Як UI/UX дизайнер у CreativeForge Studio, я займався створенням дизайну для\nдодатків з управління проєктами, спортивних платформ та новинних порталів. Мої\nобов’язки включали проведення досліджень користувачів, створення інтерактивних\nмакетів та розробку елементів інтерфейсу, що підвищували зручність користувачів.']}, {'position': '', 'company': 'Додаток для управління запасами', 'dates': 'Січень 2021 - Листопад 2022', 'responsibilities': ['Розробив API для деяких коефіцієнтів;', 'Працював над функціональністю та дизайном статистики;', 'Реалізував функцію підписки за допомогою Stripe;', 'Переробив сторінку

# Get education (institution, specialization, graduation year)

1. Search keyword: locate the ‘освіта’ keyword in the text.
2. Define education pattern: use regex to match institutions, specializations, and graduation years.
3. Extract data: for each match, structure the data with institution, specialization, and graduation year fields.
4. Format results: compile all education data into a structured list.

### TODO:

Think about making it more flexible.

In [10]:
def extract_education(text):
    education = []

    start_idx = re.search(r'освіта', text, re.IGNORECASE)
    if not start_idx:
        return None

    start_idx = start_idx.end()
    text_section = text[start_idx:].strip()

    edu_pattern = re.compile(
        r"([A-Za-zА-Яа-яІЇҐЄіїґє\s,]+)\n(.*?)\n(\d{4})",
        re.IGNORECASE
    )

    matches = edu_pattern.findall(text_section)

    for match in matches:
        institution = match[0].strip()
        specialization = match[1].strip()
        graduation_year = match[2].strip()

        education.append({
            'institution': institution,
            'specialization': specialization,
            'graduation_year': graduation_year
        })

    return education

In [11]:
resume_text = """
ОСВІТА
Київський національний університет імені Тараса Шевченка
бакалавр з комп'ютерних наук
2022

Харківський політехнічний інститут
магістр інженерії
2024
"""

# Витягнення інформації про освіту
education_details = extract_education(resume_text)
print(f"Final education details: {education_details}")

Final education details: [{'institution': 'Київський національний університет імені Тараса Шевченка', 'specialization': "бакалавр з комп'ютерних наук", 'graduation_year': '2022'}, {'institution': 'Харківський політехнічний інститут', 'specialization': 'магістр інженерії', 'graduation_year': '2024'}]


# Get skills (list of technical and soft skills)

1. Search for skill keywords: locate sections containing “skills” or “technologies” keywords.
2. Define skill pattern: use regex to match individual lines as skills, especially after bullet points.
3. Remove duplicates: add each skill to a set to ensure unique entries.
4. Format Results: convert the set back to a list and capitalize each skill.

In [12]:
def extract_skills(text):
    skills = set()

    skills_section = re.search(r'(навички|технології)', text, re.IGNORECASE)
    if not skills_section:
        return None

    start_idx = skills_section.end()
    text_section = text[start_idx:].strip()

    skill_pattern = re.compile(r'[\n-•]\s*([\w\s]+)')
    matches = skill_pattern.findall(text_section)

    for match in matches:
        skill = match.strip().capitalize()
        if skill and skill not in skills:
            skills.add(skill)

    return list(skills)

In [13]:
resume_text = """
Софт навички
- Комунікація та співпраця в міжфункціональних командах
- Управління часом і багатозадачність в умовах стиснутих термінів
- Адаптивність та готовність вивчати нові тренди та інструменти дизаи ну
Технічні навички
- Figma, Sketch, Adobe XD, InVision, Marvel, Balsamiq, Axure
- володіння HTML5, CSS3, JavaScript для прототипування та створення інтерактивних компонентів
- Розуміння основ фронтенд-розробки для ефективної співпраці з інженерами
- Досвід створення дизаи н-систем для забезпечення послідовності та ефективності на різних платформах
- Проведення досліджень користувачів, створення ваи рфреи мів та прототипів для орієнтованого на користувача дизаи ну
- Компетенція у проведенні тестування користувачів та аналізі зворотного зв'язку для покращення дизаи ну
"""

skills = extract_skills(resume_text)
print("Знайдені навички:", skills)

Знайдені навички: ['Створення ваи рфреи мів та прототипів для орієнтованого на користувача дизаи ну', 'Javascript для прототипування та створення інтерактивних компонентів', 'Balsamiq', 'Комунікація та співпраця в міжфункціональних командах', 'Систем для забезпечення послідовності та ефективності на різних платформах', 'Досвід створення дизаи н', 'Adobe xd', 'Управління часом і багатозадачність в умовах стиснутих термінів', 'Figma', 'Css3', 'Axure', 'Язку для покращення дизаи ну', 'Marvel', 'Sketch', 'Розробки для ефективної співпраці з інженерами', 'Компетенція у проведенні тестування користувачів та аналізі зворотного зв', 'Володіння html5', 'Invision', 'Розуміння основ фронтенд', 'Адаптивність та готовність вивчати нові тренди та інструменти дизаи ну\nтехнічні навички', 'Проведення досліджень користувачів']


# Parse resume

1. Check input type: confirm the input text is a string -- error if not.
2. Initialize NLP: process the text using spaCy to capture named entities.
3. Aggregate results: call each extraction function and compile results in JSON format.

In [14]:
def parse_resume(text):
    # Ensure input is a string
    if not isinstance(text, str):
        raise TypeError("Input text must be a string.")

    # Process with spaCy NLP model for name extraction
    doc = nlp(text)
    return {
        "name": extract_name_ukrainian(doc),
        "contact_info": extract_contact_info(text),
        "experience": extract_experience(text),
        "education": extract_education(text),
        "skills": extract_skills(text)
    }


> The specification states that if the model cannot find certain data, the value of this field should be set to `None`. However, in Python, the value `None` is automatically converted to `null` when converting to JSON using `json.dumps()` (this is standard behavior of the `json` library, so I'm not sure if it was necessary to additionally return `None` after conversion, since the value `null` in JSON corresponds to `None` in Python.

# Test on one resume

Demonstrates how to read and parse a single resume file (in PDF format) to show the initial parsing results.

In [15]:
text = read_pdf(file_path)
parsed_data = parse_resume(text)
parsed_json = json.dumps(parsed_data, ensure_ascii=False, indent=4)
print("\nParsed Resume Data:\n", parsed_json)

# Save the output as JSON
with open("resume_output.json", "w", encoding="utf-8") as file:
    file.write(parsed_json)


Parsed Resume Data:
 {
    "name": "Ярослав П",
    "contact_info": {
        "phones": null,
        "emails": null
    },
    "experience": [
        {
            "position": "UI Дизайнер",
            "company": "TechVega Solutions",
            "dates": "Червень 2022 - Дотепер",
            "responsibilities": [
                "У ціи ролі я відповідаю за повнии цикл дизаи ну для різноманітних цифрових\nпродуктів. Заи маюся аналізом потреб користувачів, створенням інтерфеи сів та\nітерацією дизаи нів на основі відгуків. Проєкти, над якими я працював, включають\nдодатки для систем бронювання, платформу для онлаи н-освіти та CRM-систему для\nмалого бізнесу.\n Провів юзабіліті-тестування та інтерв'ю з користувачами, щоб отримати\nінсаи ти для покращення систем бронювання.\n Розробляв каркаси, прототипи та фінальні високоякісні дизаи ни в Figma та\nAdobe XD для освітньої платформи, забезпечуючи легкість навігації та\nінтуї тивність інтерфеи су.\n Створив та підтримував масштабован

In [16]:
print("Resume Text:\n", text)

Resume Text:
 Ярослав П
UX/UI Designer | Product
Designer
Маю значнии досвід роботи в UX/UI дизаи ні, що включає створення зручних
інтерфеи сів для різноманітних цифрових продуктів, таких як освітні платформи,
CRM-системи, додатки для управління проєктами та системи бронювання.
Спеціалізуюся на проведенні досліджень користувачів, розробці каркасів і
прототипів, а також оптимізації дизаи ну на основі відгуків. Володію навичками
співпраці з розробниками для впровадження адаптивних рішень, що підвищують
доступність і зручність користування. Маю досвід у створенні дизаи н-систем, які
забезпечують єдинии стиль і полегшують подальшу розробку продуктів.
Освіта
2010-2014
Бакалавр з цифрового дизаи ну та комунікаціи
Львівськии Університет Дизаи ну
2016-2018
Магістр з цифрового дизаи ну та комунікаціи
Львівськии Університет Дизаи ну
Софт навички
 Комунікація та співпраця в міжфункціональних командах
 Управління часом і багатозадачність в умовах стиснутих термінів
 Адаптивність та готовність в

# Parse all resumes from the folder


In [17]:
def process_files_in_folder(folder_path):
    resumes = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith('.pdf'):
            text = read_pdf(file_path)
        elif file_name.endswith('.docx'):
            text = read_docx(file_path)
        elif file_name.endswith('.txt'):
            text = read_txt(file_path)
        else:
            continue

        resume_data = parse_resume(text)
        if resume_data:
            resumes.append(resume_data)

    return resumes

def write_to_json_file(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump({"resumes": data}, file, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    resumes_data = process_files_in_folder(folder_path)
    output_file = "output.json"
    write_to_json_file(resumes_data, output_file)
    print(f"Saved in {output_file}")

Saved in output.json


# Made by
[Vira Vakhovska](https://drive.google.com/file/d/1y9_qzcCZsySWdsOCmltedlA59ART2tr8/view?usp=sharing)